In [ ]:
!git clone https://github.com/zheliu17/nanoRecSys.git
%pip install -q -e ./nanoRecSys


# In fact, we don't need psutil. force-reinstall to trigger colab restart
%pip install --force-reinstall psutil=={psutil.__version__}
print("Installation complete. Please restart runtime...")

In [ ]:
import wandb
import nanoRecSys.data
import nanoRecSys.train

wandb.login()

nanoRecSys.data.build_dataset.process_data()
nanoRecSys.data.splits.create_user_time_split()

In [ ]:
class Args:
    mode = "retriever"
    epochs = 5
    batch_size = 4096
    lr = 4e-2
    num_workers = 2
    build_embeddings = True


nanoRecSys.train.main(Args)

In [ ]:
import nanoRecSys.training.mine_negatives

nanoRecSys.training.mine_negatives.main(batch_size=1024, top_k=15, skip_top=0)


class Args:
    mode = "ranker"
    epochs = 1
    limit_train_batches = 0.5
    batch_size = 2048
    explicit_neg_weight = 4
    random_neg_ratio = 0.01

    lr = 1e-3
    item_lr = 0
    num_workers = 2


nanoRecSys.train.main(Args)

In [ ]:
from nanoRecSys.eval.offline_eval import OfflineEvaluator

evaluator = OfflineEvaluator(1024)
results = evaluator.eval_popularity()

df = evaluator.formatted_results(results)
df

In [ ]:
results = evaluator.eval_ranker()

df = evaluator.formatted_results(results)
df